# Evaluation

In this experiment we evaluate which of the algorithms performs best for our problem.


In [2]:
# correct working directory.
# This is necessary for imports because the notebook is not in the main folder of the project. 
if not "working_directory_corrected" in vars():
    %cd ..
    working_directory_corrected = True


import pandas as pd

from evaluation.timed_experiment import Timed_Experiment
from algorithms.heuristic_search import Heuristic_Search
from algorithms.depth_first_search import Depth_First_Search
from algorithms.dijkstra import Dijkstra


# load dataset
from data.dataset import Dataset

c:\Users\frank\Documents\Teaching\LU\Planning and Optimization LU - Material\Planning Example Project\planning_example_project\environment\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\frank\Documents\Teaching\LU\Planning and Optimization LU - Material\Planning Example Project\planning_example_project


## Procedure

The cell below runs all algorithms and records data.

The following algorithms are run as a consequence of previous experiments:
- Depth First Search based on the original graph.
- Heuristic Search based on the original graph.
- Dijkstra's algorithm in conjunction with the layout algorithm "drl" in variation 1 and scale factor of 10.

Each run will be comprised of 10000 planning problems. Each planning algorithm will be initialized with the same random seed. Since we used the seed 42 in previous experiments we will choose a different one here, to avoid bias towards the problems produced by that seed.

From each experiment we will collect the following data:
- *average_time*: The average time they took to solve each problem.
- *nr_extended*: The number of nodes that were extended.
- *time per extension*: The average time required for one extension.
We will also record these three values for failed and successfull runs.

After running the experiments we will use these values to discuss the algorithms and select which one would be most suitable. For suitability, their average run time is the selection criterium.

In [5]:

#experiment configuration


runs = 10000
algorithm_names = [None, None, "drl"]
variations = [None,None,1]
scale_factors = [None,None,10]
planners = [Depth_First_Search, Heuristic_Search, Dijkstra] 

# test planners
for index in range(len(planners)):
    planner = planners[index]
    algorithm_name = algorithm_names[index]
    variation = variations[index]
    scale_factor = scale_factors[index]

    
    # load graph
    file_name = f"if_{algorithm_name}_{variation}"
    dataset = Dataset()
    graph = dataset.load_graph()
    if algorithm_name is not None:
        dataset.convert_to_spatial(graph,file_name, scale_factor=scale_factor)


    print(f"\nrunning {planner} on algorithm {algorithm_name} with variation {variation}")
    experiment = Timed_Experiment(graph, planner, runs, random_seed=7, verbose = False)
    experiment.run()
    print("Average Time: ", int(experiment.get_average_time()), "ns")
    print("Nr Extended: ", experiment.get_nr_extensions())
    print("Time per extension: ", int(experiment.get_average_extension_time()), "ns")
    
    print("Nr successful runs:", experiment.get_nr_successful_runs())	
    print("Average Successful Time: ", int(experiment.get_average_successful_time()), "ns")	 
    print("Nr Successful Extensions: ", experiment.get_nr_successful_extensions())	
    print("Time per successful extension: ", int(experiment.get_average_successful_extension_time()), "ns")	
    
    print("Nr unsuccessful runs:", experiment.get_nr_unsuccessful_runs())	
    print("Average Unsuccessful Time:", int(experiment.get_average_unsuccessful_time()), "ns")	 
    print("Nr Failed Extensions: ", experiment.get_nr_unsuccessful_extensions())	
    print("Time per failed extension: ", int(experiment.get_average_unsuccessful_extension_time()), "ns")	



running <class 'algorithms.depth_first_search.Depth_First_Search'> on algorithm None with variation None
Average Time:  72269706 ns
Nr Extended:  328047519
Time per extension:  2203 ns
Nr successful runs: 1381
Average Successful Time:  63680977 ns
Nr Successful Extensions:  36415112
Time per successful extension:  2415 ns
Nr unsuccessful runs: 8619
Average Unsuccessful Time: 73645856 ns
Nr Failed Extensions:  291632407
Time per failed extension:  2176 ns

running <class 'algorithms.heuristic_search.Heuristic_Search'> on algorithm None with variation None
Average Time:  110705766 ns
Nr Extended:  328047519
Time per extension:  3374 ns
Nr successful runs: 1381
Average Successful Time:  94847983 ns
Nr Successful Extensions:  36415112
Time per successful extension:  3596 ns
Nr unsuccessful runs: 8619
Average Unsuccessful Time: 113246617 ns
Nr Failed Extensions:  291632407
Time per failed extension:  3346 ns

running <class 'algorithms.dijkstra.Dijkstra'> on algorithm drl with variation 1


### Results

The below table is the result of running the experiment on a Lenovo ThinkPad E14 Gen 6, standard configuration.
While running, the notebook has been left alone, to minimize the influence of other applications.

| Metric | Depth First Search | Heuristic Search | Djkstra's Algorithm |
| --- | --- | --- | --- |
| Average Time (All)| 72 ms  | 110 ms  | 123 ms |
| Average Time (Success)| 64 ms  | 95 ms  | 102 ms |
| Average Time (Fail)| 74 ms  | 113 ms  | 127 ms |
| Nr Extended (All)| 328.047.519 | 328.047.519 | 327.444.468 |
| Nr Extended (Success)| 36.415.112 | 36.415.112 | 35.813.060 |
| Nr Extended (Fail)| 291.632.407 | 291.632.407 | 291.631.408 |
| Time per extension (All)| 2203  ns | 3374 ns  | 3782 ns |
| Time per extension (Success)| 2415  ns  | 3596 ns  | 3944 ns |
| Time per extension (Fail)| 2176  ns  | 3346 ns  | 3763 ns |


### Discussion

Depth first search performed significantly better that the two other algorithms in all cases. 

While Dijkstra's algorithm was able to dredue on teh number of extended nodes in successful cases, it did not speed up the overall search time sufficiently to matter.
The reason for the better performance of Depth first search is likely the higher computational overhead of the other algorithms (see time per extension) and the fact that the problem search space didn't allow them to counteract this overhead by finding a result in less steps.

It should be noted that this experiment only compared the search times and ignored the time taken to run the layout algorithm. Had an informed algorithm outperformed the uninformed one, this time would have had to be taken into account.

### Conclusion

We recommend Depth First Search for this problem as it outperformed all other algorithms with regard to average run time.